<a href="https://colab.research.google.com/github/ssrbazpur/BigQuery/blob/master/Model_Building_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import sys
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/Data Science/nyc-taxi-265120-baf9a3e4cf9b.json"
%reload_ext google.cloud.bigquery

In [0]:

project_id = 'nyc-taxi-265120' #@param{type:"string"}
from google.cloud import bigquery
#from google.cloud import bigquery_storage_v1beta1
# Create BigQuery client
client = bigquery.Client(project = project_id)
# Create BigQuery storage client
#bq_storage_client = bigquery_storage_v1beta1.BigQueryStorageClient()


<h3> Collecting Data Prepared </h3>

In [0]:
#2018 DATA
query = """
with wd as (
    SELECT 
        cast(year as STRING) as year,
        EXTRACT (DAYOFYEAR FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP)) AS daynumber, 
        MIN(EXTRACT (DAYOFWEEK FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP))) dayofweek,
       MIN(temp) temp,  MIN(min) mintemp, MAX(max) maxtemp, MAX(IF(prcp=99.99,0,prcp)) rain
    FROM `bigquery-public-data.noaa_gsod.gsod2018`
    WHERE stn='725030'   --station id 725030=LaGuardia
    GROUP BY 1,2 
  
  -- TAXI DATA
  ),
TD AS(

SELECT  zone_id, time, pickups ,CAST(EXTRACT (YEAR from time) AS STRING) AS year,EXTRACT (DAYOFYEAR from time) AS daynumber,EXTRACT (HOUR from time) AS hour from `nyc-taxi-265120.hello.2018Data` ),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)


SELECT 
td.year,
points.zone_id,

td.time,
td.daynumber,
td.hour,
cast(wd.dayofweek as STRING) as dayofweek, 
   wd.temp,  wd.mintemp, 
    wd.maxtemp,
    wd.rain,
 td.pickups,

   ST_X(p) as longitude,
  ST_Y(p) as latitude
 
  FROM wd, td INNER JOIN points ON points.zone_id=td.zone_id
  where wd.year = td.year AND
  wd.daynumber = td.daynumber
  group by year,zone_id,time,daynumber,hour,pickups,longitude,latitude,dayofweek, temp,mintemp, maxtemp, rain

"""
df1 = client.query(query).to_dataframe()


In [0]:
#2019 DATA
query = """
with wd as (
    SELECT 
        cast(year as STRING) as year,
        EXTRACT (DAYOFYEAR FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP)) AS daynumber, 
        MIN(EXTRACT (DAYOFWEEK FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP))) dayofweek,
        MIN(temp) temp,MIN(min) mintemp, MAX(max) maxtemp, MAX(IF(prcp=99.99,0,prcp)) rain
    FROM `bigquery-public-data.noaa_gsod.gsod2019`
    WHERE stn='725030'   --station id 725030=LaGuardia
    GROUP BY 1,2 
  
  -- TAXI DATA
  ),
TD AS(

SELECT  zone_id, time, pickups ,CAST(EXTRACT (YEAR from time) AS STRING) AS year,EXTRACT (DAYOFYEAR from time) AS daynumber,EXTRACT (HOUR from time) AS hour from `nyc-taxi-265120.hello.test_table` ),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)


SELECT 
td.year,
points.zone_id,

td.time,
td.daynumber,
td.hour,
cast(wd.dayofweek as STRING) as dayofweek, 
   wd.temp,
    wd.mintemp, 
    wd.maxtemp,
    wd.rain,
    
 td.pickups,

    ST_X(p) as longitude,
  ST_Y(p) as latitude
 
  FROM wd, td INNER JOIN points ON points.zone_id=td.zone_id
  where wd.year = td.year AND
  wd.daynumber = td.daynumber
  group by year,zone_id,time,daynumber,hour,pickups,longitude,latitude,dayofweek,temp, mintemp, maxtemp, rain

"""
df2 = client.query(query).to_dataframe()


In [0]:
#DROP p
df1.drop('P', axis=1, inplace=True)
df2.drop('P', axis=1, inplace=True)

In [16]:
df2.head()

,year,zone_id,time,daynumber,hour,dayofweek,temp,mintemp,maxtemp,rain,pickups,longitude,latitude
0,2019,1,2019-01-01 00:00:00+00:00,1,0,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183
1,2019,1,2019-01-01 01:00:00+00:00,1,1,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183
2,2019,1,2019-01-01 03:00:00+00:00,1,3,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183
3,2019,1,2019-01-01 04:00:00+00:00,1,4,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183
4,2019,1,2019-01-01 07:00:00+00:00,1,7,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183


In [0]:
df2.drop('time', axis=1, inplace=True)
df1.drop('time', axis=1, inplace=True)

In [0]:
df2["zone_id"] = df2["zone_id"].astype(int)
df2["dayofweek"] = df2["dayofweek"].astype(int)

In [0]:
df1["zone_id"] = df1["zone_id"].astype(int)
df1["dayofweek"] = df1["dayofweek"].astype(int)

In [0]:
finaldf=pd.concat([df1, df2])

In [21]:
finaldf.head()

,year,zone_id,daynumber,hour,dayofweek,temp,mintemp,maxtemp,rain,pickups,longitude,latitude
0,2018,1,1,0,2,12.6,8.1,21.9,0.0,0.0,-74.174002,40.69183
1,2018,1,1,1,2,12.6,8.1,21.9,0.0,0.0,-74.174002,40.69183
2,2018,1,1,2,2,12.6,8.1,21.9,0.0,0.0,-74.174002,40.69183
3,2018,1,1,5,2,12.6,8.1,21.9,0.0,0.0,-74.174002,40.69183
4,2018,1,1,19,2,12.6,8.1,21.9,0.0,0.0,-74.174002,40.69183


In [0]:
# import pandas_gbq as gbq
# gbq.to_gbq(finaldf, 'hello.exploredata', 'nyc-taxi-265120', if_exists='replace')

1it [00:38, 38.68s/it]


In [27]:
finaldf.info()
finaldf["year"] = finaldf["year"].astype(int)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1852109 entries, 0 to 925924
Data columns (total 12 columns):
year         object
zone_id      int64
daynumber    int64
hour         int64
dayofweek    int64
temp         float64
mintemp      float64
maxtemp      float64
rain         float64
pickups      float64
longitude    float64
latitude     float64
dtypes: float64(7), int64(4), object(1)
memory usage: 183.7+ MB


In [0]:
y=finaldf['pickups']
X=finaldf.drop('pickups', axis=1)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [0]:
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [31]:
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
       verbose=True) # Change verbose to True if you want to see it train

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:39:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:120.541	validation_1-rmse:121.115
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:114.113	validation_1-rmse:114.721
[2]	validation_0-rmse:108.262	validation_1-rmse:108.916
[3]	validation_0-rmse:103.114	validation_1-rmse:103.798
[4]	validation_0-rmse:98.6201	validation_1-rmse:99.3392
[5]	validation_0-rmse:94.8302	validation_1-rmse:95.5736
[6]	validation_0-rmse:91.6211	validation_1-rmse:92.3946
[7]	validation_0-rmse:88.9611	validation_1-rmse:89.7551
[8]	validation_0-rmse:86.6119	validation_1-rmse:87.4253
[9]	validation_0-rmse:84.5486	validation_1-rmse:85.3886
[10]	validation_0-rmse:82.5019	validation_1-rmse:83.3482
[11]	validation_0-rmse:80.9515	validation_1-rmse:81.8137
[12]	validation_0-rmse:79.6552	validatio

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

KeyboardInterrupt: ignored

TRAIN 2019 DATA ONLY

In [0]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925925 entries, 0 to 925924
Data columns (total 13 columns):
year         925925 non-null object
zone_id      925925 non-null object
time         925925 non-null datetime64[ns, UTC]
daynumber    925925 non-null int64
hour         925925 non-null int64
dayofweek    925925 non-null object
temp         925925 non-null float64
mintemp      925925 non-null float64
maxtemp      925925 non-null float64
rain         925925 non-null float64
pickups      925925 non-null float64
longitude    925925 non-null float64
latitude     925925 non-null float64
dtypes: datetime64[ns, UTC](1), float64(7), int64(2), object(3)
memory usage: 91.8+ MB


In [0]:
df2.head()

,year,zone_id,time,daynumber,hour,dayofweek,temp,mintemp,maxtemp,rain,pickups,longitude,latitude
0,2019,1,2019-01-01 00:00:00+00:00,1,0,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183
1,2019,1,2019-01-01 01:00:00+00:00,1,1,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183
2,2019,1,2019-01-01 03:00:00+00:00,1,3,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183
3,2019,1,2019-01-01 04:00:00+00:00,1,4,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183
4,2019,1,2019-01-01 07:00:00+00:00,1,7,3,51.1,35.1,60.1,1.39,0.0,-74.174002,40.69183


In [0]:
#DROP p
df2.drop('mintemp', axis=1, inplace=True)
df2.drop('maxtemp', axis=1, inplace=True)
df2.drop('latitude', axis=1, inplace=True)
df2.drop('longitude', axis=1, inplace=True)
df2.drop('time', axis=1, inplace=True)


In [0]:
df2.head()

,year,zone_id,daynumber,hour,dayofweek,temp,rain,pickups
0,2019,1,1,0,3,51.1,1.39,0.0
1,2019,1,1,1,3,51.1,1.39,0.0
2,2019,1,1,3,3,51.1,1.39,0.0
3,2019,1,1,4,3,51.1,1.39,0.0
4,2019,1,1,7,3,51.1,1.39,0.0


In [0]:
df2["zone_id"] = df2["zone_id"].astype(int)
df2["dayofweek"] = df2["dayofweek"].astype(int)

In [0]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925925 entries, 0 to 925924
Data columns (total 8 columns):
year         925925 non-null object
zone_id      925925 non-null int64
daynumber    925925 non-null int64
hour         925925 non-null int64
dayofweek    925925 non-null int64
temp         925925 non-null float64
rain         925925 non-null float64
pickups      925925 non-null float64
dtypes: float64(3), int64(4), object(1)
memory usage: 56.5+ MB


In [0]:
df2["year"] = df2["year"].astype(int)

In [0]:

y=df2['pickups']
X=df2.drop('pickups', axis=1)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
a=rf.predict(X_test)

In [0]:
from sklearn.metrics import mean_squared_error
print('The rmse of prediction is:', mean_squared_error(y_test, a) ** 0.5)

The rmse of prediction is: 15.049406648468416


In [0]:
print(int(rf.predict([[2017,40,1,0,4,-100,6]])))

4


In [0]:
 X_test.head()

,year,zone_id,daynumber,hour,dayofweek,temp,rain
837074,2019,187,135,21,4,53.8,0.21
238577,2019,99,39,17,6,45.0,0.33
485529,2019,40,79,10,4,43.7,0.00
612843,2019,168,99,20,3,47.0,0.02
602316,2019,248,97,13,1,55.6,0.00


In [0]:
y_test.head()

837074    0.0
238577    0.0
485529    4.0
612843    1.0
602316    0.0
Name: pickups, dtype: float64

In [0]:
round_off_values = np.round_(a) 

In [0]:
import lightgbm as lgb
print("Fitting...")
parameters = gbm.fit( X_train, y_train )
print("Fitted...")